# Introduction
I will be using dataset from [Friends_series](https://raw.githubusercontent.com/uragirii/Friends-Generator/master/Data/Friends_Transcript.txt) to make an RNN based model to generate new scripts for the project.

# About Friends Series
It was an American sitcom that aired from 1994-2004 with 10 seasons produced. It is one of my favorite shows so thought of making a model to generate new scripts for the series. Further info can be seen [from](https://en.wikipedia.org/wiki/Friends)

![friends](https://pmcvariety.files.wordpress.com/2020/02/friends.jpg?w=681&h=383&crop=1)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import helper

# Loading the Dataset

In [ ]:
data_dir = tf.keras.utils.get_file('Friends_Transcript.txt', 'https://raw.githubusercontent.com/uragirii/Friends-Generator/master/Data/Friends_Transcript.txt')
with open(data_dir) as f:
    text = f.readlines()
text = [x.strip() for x in text] 
print(text[:20])

# Analyzing the Script data
I have taken insights from Shakespeare Dataset which can be found on [tensorflow's official website](https://www.tensorflow.org/tutorials/text/text_generation)

In [ ]:
text = open(data_dir, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [ ]:
#Vocablary can be used to get the unique texts in the dataset
vocab = sorted(set(text))
print('The Number of Unique words : {}'.format(len(vocab)))

# Text processing

# Vectorize the Dataset

In [ ]:
char_to_idx = {u:i for i, u in enumerate(vocab)}
idx_to_char = np.array(vocab)

text_as_int = np.array([char_to_idx[c] for c in text])

***Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).***

In [ ]:
print('{')
for char,_ in zip(char_to_idx, range(25)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_idx[char]))
print('  ...\n}')

In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

# Model developement
***The prediction task***

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

***Create training examples and targets***

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [ ]:
#Maximum length of sequence to be taken as a single input in characters
seq_length = 100
examples_per_epochs = len(text)//(seq_length + 1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx_to_char[i.numpy()])

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx_to_char[item.numpy()])))

For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx_to_char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx_to_char[target_example.numpy()])))

**Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "T" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.**

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx_to_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx_to_char[target_idx])))

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

***Now we start building the model***

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

**Model Layout**
1. Sequential
2. Embedding
3. GRU
4. Dense

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

**Process flow from the above codes can be visualized here!!**
![view](https://www.tensorflow.org/tutorials/text/images/text_generation_training.png)

In [ ]:
model.summary()

**Shape of Output**

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# Predictions 
To make the predictions of next line in the dataset, we first sample the output distribution to get the actual character indices

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

**Let's predict the text after Decoding the sampled_indices**

In [ ]:
print("Input: \n", repr("".join(idx_to_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx_to_char[sampled_indices ])))

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True )
model.compile(optimizer = 'adam', loss = loss)

In [ ]:
#saving the model
checkpoint_dir = '../training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
import shutil
shutil.rmtree(checkpoint_dir)

In [ ]:
epochs = 30
history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(history.history['loss'], label='Loss')

plt.legend()
plt.title('Metrics estimations')

**Generate text**

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 1000

  # Converting our start string to numbers (vectorizing)
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1.0

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx_to_char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Chandler: "))